In [1]:
from gps import GPS

gps = GPS('COM8')

In [2]:
current_position = gps.get_current_location()
print(current_position)
gps.save_data2folder()

{'latitude': -12.463007, 'longitude': 130.839848}


In [3]:
from db_querier import DbQuerier

db_querier = DbQuerier(gps)

Data read from C:/Users/giaco/Git_Repositories/aCentauri/can-msg-api/db/.env.


In [4]:
v = db_querier.query_velocity()
soc = db_querier.query_soc()
print(soc)

db_querier.save_data2folder()

                       time      SoC   latitude   longitude
0 2023-10-12 14:02:50+09:30  0.14008 -12.463007  130.839848


In [5]:
from route import Route
from route_api import RouteAPI

# current_position = {'longitude': 134.141402,
#                     'latitude':-18.991183}
final_position = {'longitude': 133.8750197,
                  'latitude': -23.7090616}
delta_spacing = 5000.0 # in meters
number_sites = 150

route = Route()
api_route = RouteAPI(route)

api_route_df = api_route.cut_route_data(current_position, number_sites=number_sites, print_is_requested=True)
# api_route_df = api_route.cut_route_data(current_position, number_sites=number_sites, delta_spacing=delta_spacing, print_is_requested=True)
# api_route_df = api_route.cut_route_data(current_position=current_position, final_position=final_position, number_sites=number_sites, print_is_requested=False)
# print(route.find_closest_row(current_position))

# route.insert_to_control_stops()

Nearest index in csv file: 5
Nearest index in dataframe: 3
      longitude   latitude  altitudeSmoothed  inclinationSmoothed  \
0    130.842479 -12.465529         38.698616            -0.003396   
1    130.993006 -12.471555         42.303277             0.009456   
2    131.072790 -12.621587         25.312995             0.005977   
3    131.123128 -12.792734         36.802667             0.001552   
4    131.118268 -12.965797         98.689180            -0.003862   
..          ...        ...               ...                  ...   
145  138.246628 -34.297349          9.838988             0.000329   
146  138.355335 -34.452633         14.355906            -0.002033   
147  138.502745 -34.588656         13.693952            -0.002058   
148  138.589423 -34.752646         10.871189             0.001063   
149  138.600712 -34.928189         64.449372            -0.011632   

      cumDistance  maxSpeed         theta  timeAtMaxSpeed  cumTimeAtMaxSpeed  \
0    2.744025e+01        40  7.9

In [6]:
# route_data = route.get_route_data

# def cumTimeAtMeanVelocity(velocity:float) -> pd.DataFrame:
#     """ vel in km/h """
#     vec = route_data['distance'] / velocity * 3.6
#     data = route_data.copy()
#     data['cumTimeAtMeanVelocity'] = vec.cumsum()
#     return data

# print(cumTimeAtMeanVelocity(velocity=40))

In [9]:
import pandas as pd
from Solcast_executer import SolcastExecuter
solcast_executer = SolcastExecuter()

# TODO TIMEOUT ERROR BUG
# current_position = {'longitude': 130.9064683,
#                     'latitude': -12.4479361}
# raw_df = solcast_executer.get_forecast(current_position)
raw_df = solcast_executer.get_forecasts(api_route_df, hours_in_advance=48)
print(raw_df)

                                        tt   gh     ff   dd    rh   rr
cumDistance  time                                                     
2.744025e+01 2023-10-18 10:45:00+09:30  32  868   5.40   68  40.8  0.0
             2023-10-18 11:00:00+09:30  32  901   5.76   53  39.7  0.0
             2023-10-18 11:15:00+09:30  33  929   6.48   40  38.7  0.0
             2023-10-18 11:30:00+09:30  33  954   7.20   30  37.7  0.0
             2023-10-18 11:45:00+09:30  33  973   7.92   19  37.2  0.0
...                                     ..  ...    ...  ...   ...  ...
3.026918e+06 2023-10-20 09:45:00+09:30  20  764  16.20  306  52.4  0.0
             2023-10-20 10:00:00+09:30  20  803  15.48  307  52.4  0.0
             2023-10-20 10:15:00+09:30  20  840  15.12  308  52.3  0.0
             2023-10-20 10:30:00+09:30  20  866  14.76  309  52.2  0.0
             2023-10-20 10:45:00+09:30  20  886  14.76  309  52.0  0.0

[28950 rows x 6 columns]


In [10]:
from Meteotest_executer import MeteotestExecuter
from Meteotest_requester import MeteotestRequester
from Meteotest_parser import MeteotestParser

api_parser = MeteotestParser()
api_requester = MeteotestRequester(parser=api_parser, print_is_requested=False)
api_executer = MeteotestExecuter(requester=api_requester)

Response status from siteinfo: OK.


In [ ]:
# api_requester.get_site_info(print_is_requested=True)
# api_executer.get_all_site_id

In [ ]:
# api_requester.get_site_add(name="0", position=current_position, print_is_requested=True)
# api_executer.add_sites(api_route_df, print_is_requested=True)

In [ ]:
# api_requester.get_site_delete(585394, print_is_requested=True)
# api_executer.delete_all_sites()

In [ ]:
# site_id = 54689
# name = "404"
# position = {'latitude': 136,
#             'longitude': -30}

# api_requester.get_site_edit(site_id, name=name, position=position)

# api_route_df.index=api_executer.get_all_site_id ########TODO ADAPT DIRECTLY 
# api_executer.edit_sites(api_route_df)

In [ ]:
# raw_df = api_requester.get_solar_forecast()
# print(raw_df.head())

# raw_df = api_requester.get_solar_forecast_cloudmove()
# print(raw_df)
# raw_SF_df, _, raw_CM_df, _ = api_executer.get_new_forecasts()

# print(raw_SF_df.head(), raw_CM_df.head())

In [11]:
sites_df = api_requester.get_current_sites

from preprocessor import Preprocessor
preprocessor = Preprocessor()
hours_in_advance = None
preprocessed_df, preprocessed_df = preprocessor.preprocess(api_route_df, sites_df, raw_df, hours_in_advance)
# preprocessor.save_data2folder()
print(preprocessed_df)

                                        temperature  globalIrradiance  \
cumDistance  time                                                       
2.744025e+01 2023-10-18 10:45:00+09:30           35               868   
             2023-10-18 11:00:00+09:30           35               901   
             2023-10-18 11:15:00+09:30           36               929   
             2023-10-18 11:30:00+09:30           36               954   
             2023-10-18 11:45:00+09:30           36               973   
...                                             ...               ...   
3.026918e+06 2023-10-20 09:45:00+09:30           20               764   
             2023-10-20 10:00:00+09:30           23               803   
             2023-10-20 10:15:00+09:30           23               840   
             2023-10-20 10:30:00+09:30           23               866   
             2023-10-20 10:45:00+09:30           23               886   

                                        windSpeed 

In [12]:
from opt_reader import OptReader

opt_reader = OptReader()
DP_data = opt_reader.get_optimal_data

Data read from G:/Drive condivisi/AlphaCentauri/SolarCar_22 23/6. Strategy & Simulation/ss_online_data/DP_optimal/CSV/20231018_093437_DP.csv.


In [16]:
import pandas as pd
import dash
import constants
import plotly.graph_objects as go
from dash import Dash, html, dcc
from dash.dependencies import Input, Output, State
from functions import rad2deg

import random


control_stops_df = route.get_control_stops_data
sites_df = api_requester.get_current_sites
current_position['cumDistance'] = route.find_closest_row(current_position, print_is_requested=False)[0]['cumDistance']
current_position['cumDistance_km'] = current_position['cumDistance'] / 1000

# Get the current time
now = pd.Timestamp.now()
current_hour = now.hour
current_minute = now.minute
# Create hour and minute options based on constraints
hour_options = [{'label': str(i), 'value': i} for i in range(current_hour, 18)]
minute_options = [{'label': str(i), 'value': i} for i in range(current_minute, 60)] if current_hour == now.hour else [{'label': str(i), 'value': i} for i in range(60)]


app = Dash(__name__)

# aCe images
url_image = 'https://avatars.githubusercontent.com/u/115261983?s=280&v=4'

## Time
# Slider
height_slider = 1000
unique_times = preprocessed_df.index.get_level_values('time').unique()
slider_marks = {i: time.strftime('%H:%M') for i, time in enumerate(unique_times)}

# Dropdown
unique_days = unique_times.map(lambda x: x.strftime('%d %B %Y')).unique().to_list()

# Update time
update_sec = 40

## Map
route_data = route.get_route_data
max_cumDistance_km = route_data['cumDistance'].max() / 1000
row_current_cumDistance, _ = route.find_closest_row(current_position)
min_lng = route_data['longitude'].min()
min_lat = route_data['latitude'].min()
max_lng = route_data['longitude'].max()
max_lat = route_data['latitude'].max()
middle_lat = (max_lat + min_lat) / 2
middle_lng = (max_lng + min_lng) / 2

## Plot
x_cumDistance_km = route_data['cumDistance'] / 1000
x_cumDistance_str = 'Cumulative Distance (km)'

grid_color = 'lightgray'
box_color = 'black'

height_plot = 280

# Control stops, start, and finish lines
control_stops_df = route.get_control_stops_data

control_stops = control_stops_df.iloc[1:-1]
cs_vertical_lines = []
for _, row in control_stops.iterrows():
    x_value = row['cumDistance'] / 1000
    cs_vertical_lines.append(
        dict(type='line', x0=x_value, x1=x_value, y0=0, y1=1, yref='paper', line=dict(color='orange', dash='dash')) #, hovertext=row['town'] + ': ' + row['location'], hoverinfo='text'
    )
#TODO CHECK HOVERINFO
start_finish = control_stops_df.iloc[[0, -1]]
start_finish_lines = []
for _, row in start_finish.iterrows():
    x_value = row['cumDistance'] / 1000
    start_finish_lines.append(
        dict(type='line', x0=x_value, x1=x_value, y0=0, y1=1, yref='paper', line=dict(color='black', width=0.5)) #, hovertext=row['town'] + ': ' + row['location'], hoverinfo='text'
    )

current_position_line = [{
            'type': 'line',
            'x0': current_position['cumDistance_km'],
            'x1': current_position['cumDistance_km'],
            'y0': 0,
            'y1': 1,
            'yref': 'paper',
            'line': {
                'color': 'red',
                'width': 1}
            },
        ]
    
vertical_lines = cs_vertical_lines + start_finish_lines + current_position_line

# Overnight functions
def _recursive_position_finder(current_cumDistance:float, driving_time:float, cs_to_skip:int, i:int) -> pd.Series:
    """ """
    # print(i)
    # Cut data at current position (lower cut)
    cut_data = route_data.copy()

    cut_data = cut_data[cut_data['cumDistance'] >= current_cumDistance]
    cut_data = cut_data.reset_index(drop=True)

    current_time = cut_data['cumTimeAtMaxSpeedLim'][0]

    # Cut data at driving time (upper cut)
    cut_data = cut_data[cut_data['cumTimeAtMaxSpeedLim'] <= current_time + driving_time]
    max_cumDistance = cut_data['cumDistance'].max()

    # Check if the control stop dataframe is not empty
    if not control_stops.empty:
        cs_in_range_mask = (control_stops['cumDistance'] >= current_cumDistance) & (control_stops['cumDistance'] <= max_cumDistance)
        cs_in_range = cs_in_range_mask.sum()
        # print(f'cs found ahead: {cs_in_range}')
        # print(f'cs to skip: {cs_to_skip}')
    else:
        print("No control stop dataframe given")

    # Stop cases
    # Reach end of route, return last point
    if current_cumDistance >= route_data.iloc[-1]['cumDistance']:
        return route_data.iloc[-1] # return self.end_position
    
    # All control stops considered
    if cs_to_skip == cs_in_range:
        print("All control stops considered")
        return route_data.loc[route_data['cumDistance'] == max_cumDistance].iloc[0]
    
    # Stop at control stop for the night, meaning we arrive at cs between 16:30 and 17:00
    if cs_to_skip > cs_in_range:
        print("Stop at control stop for the night")
        return control_stops.loc[control_stops['cumDistance'] > current_cumDistance].iloc[cs_to_skip - 1]
    

    # Recursive call to skip control stop and reduce driving time by 30 minutes
    if cs_to_skip < cs_in_range: # Case of 0 cs in range considered
        print("--- Recursive call ---")
        return _recursive_position_finder(current_cumDistance, driving_time - 30.0*60.0, cs_to_skip + 1, i+1)
    
def update_max_speed_distance(current_position:dict, hour:int=17, minute:int=0) -> float:
    """ """
    # Subtract overnight stop start time to now
    now = pd.Timestamp.now()

    if now.hour >= hour or now.hour < 8:
        # Calculate the next overnight stop  from 8 in the morning
        driving_time = pd.Timedelta(hours=hour, minutes=minute) - pd.Timedelta(hours=8)

    else:
        driving_time = pd.Timedelta(hours=hour, minutes=minute) - pd.Timedelta(hours=now.hour, minutes=now.minute)
    
    # Call the recursive finder
    position_series = _recursive_position_finder(current_position['cumDistance'], driving_time.total_seconds(), cs_to_skip=0, i=0)

    return position_series['cumDistance']

def cumTimeAtMeanVelocity(velocity:float) -> pd.DataFrame:
    """ vel in km/h """
    vec = route_data['distance'] / velocity * 3.6
    data = route_data.copy()
    data['cumTimeAtMeanVelocity'] = vec.cumsum()
    return data


## Function to initialize plots
def initialize_plot(title, x:list=[], y:list=[]) -> go.Figure:
    return go.Figure(
        data=[go.Scatter(
            x=x,
            y=y,
            mode='lines',
            hoverinfo='none')],
        layout = go.Layout(
            margin={"r": 0, "t": 0},
            plot_bgcolor='white',
            xaxis={
                'title': x_cumDistance_str,
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color
            },
            yaxis={
                'title': title,
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color,
                'zeroline': True,
                'zerolinecolor': grid_color,
                'zerolinewidth': 0.5
            },
            shapes=vertical_lines,
            height=height_plot,
            legend={
                'x': 1,
                'y': 1,
                'xanchor': 'right',
                'yanchor': 'top'
            },
        )
    )

app.layout = html.Div([
    dcc.Interval(id='interval_update', interval=update_sec*1000, n_intervals=0),
    dcc.Store(id='store_current_position', data=current_position),
    dcc.Store(id='store_driver_stop', data={}),

    # Header with image and title
    html.Div([
        # Title
        html.H1("Strategy Interface",
                style={'textAlign': 'left', 'margin': '0 auto', 'flex': '1'}),

        # Dropdown
        dcc.Dropdown(options=[{'label': day, 'value': day} for day in unique_days],
                     value=unique_days[0],
                     id='dropdown_day_month_year',
                     style={'width': '200px', 'margin-right': '20px'}),
        
        # Last update
        html.Div([
            html.Div(id='last_update_sf',
                     style={'color': 'gray'}),
            html.Div(id='last-update-cm',
                     style={'color': 'gray'}),
            html.Div(id='last_update_sc',
                     style={'color': 'gray'}),
        ], style={'display': 'flex', 'flexDirection': 'column', 'margin-right': '20px'}),

        # Image
        html.Img(src=url_image,
                 style={'height': '100px', 'width': 'auto', 'margin-left': '20px'}),
        
    ], style={'display': 'flex', 'alignItems': 'center', 'padding-left': '20px', 'padding-right': '20px'}),


    # Container for the scouts
    html.Div([

        # Overnight stop
        html.Div([
            html.Table([
                # Header with merged cells
                html.Tr([html.Th('Next overnight stop', colSpan=2)]),
                
                # Remaining rows
                html.Tr([html.Th('At max speed', colSpan=2)]),
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='os_max_v_lat'), html.Td(id='os_max_v_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
                html.Tr([html.Td(id='os_max_v_dist_darw'), html.Td(id='os_max_v_dist_us')]),

                html.Tr([html.Th('At optimal speed', colSpan=2)]),
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='os_opt_v_lat'), html.Td(id='os_opt_v_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
                html.Tr([html.Td(id='os_opt_v_dist_darw'), html.Td(id='os_opt_v_dist_us')]),

                html.Tr([html.Th('At average speed', colSpan=2)]),
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='os_mean_v_lat'), html.Td(id='os_mean_v_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
                html.Tr([html.Td(id='os_mean_v_dist_darw'), html.Td(id='os_mean_v_dist_us')]),
            ]),

            html.Div([
                dcc.Input(id='input_cum_dist', type='number', min=current_position['cumDistance_km'], max=max_cumDistance_km, placeholder='Distance from Darwin in km'),
                html.Button('Submit', id='submit_cum_dist', disabled=True),
                html.Div(id='output_cumDistance')
            ]),
            html.Div([
                dcc.Input(id='input_lat', type='number', min=constants.GEO['latitude']['min'], max=constants.GEO['latitude']['max'], placeholder='Latitude in °'),
                dcc.Input(id='input_lng', type='number', min=constants.GEO['longitude']['min'], max=constants.GEO['longitude']['max'], placeholder='Longitude in °'),
                html.Button('Submit', id='submit_lat_lng', disabled=True),
                html.Div(id='output_lat_lng')
            ]),
        ]),

        # Driver stop
        html.Div([
            html.Table([
                # Header with merged cells
                html.Tr([html.Th('Next driver stop', colSpan=2)]),
                
                # Remaining rows
                html.Tr([html.Th('At max speed', colSpan=2)]),
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='ds_max_v_lat'), html.Td(id='ds_max_v_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
                html.Tr([html.Td(id='ds_max_v_dist_darw'), html.Td(id='ds_max_v_dist_us')]),

                html.Tr([html.Th('At optimal speed', colSpan=2)]),
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='ds_opt_v_lat'), html.Td(id='ds_opt_v_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
                html.Tr([html.Td(id='ds_opt_v_dist_darw'), html.Td(id='ds_opt_v_dist_us')]),

                html.Tr([html.Th('At average speed', colSpan=2)]),
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='ds_mean_v_lat'), html.Td(id='ds_mean_v_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
                html.Tr([html.Td(id='ds_mean_v_dist_darw'), html.Td(id='ds_mean_v_dist_us')]),
            ]),

            html.Div([
                dcc.Dropdown(
                    id='dropdown_ds_hour',
                    options=hour_options,
                    value=current_hour,
                    clearable=False
                ),
                dcc.Dropdown(
                    id='dropdown_ds_minute',
                    options=minute_options,
                    value=current_minute,
                    clearable=False
                ),
                html.Button('Submit', id='submit_time_ds', disabled=True),
                html.Div(id='output_time')
            ]),
        ]),

        # Current position and control stop
        html.Div([
            html.Table([
                # Header with merged cells
                html.Tr([html.Th('Current position', colSpan=2)]),
                
                # Remaining rows
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='current_lat'), html.Td(id='current_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance to Adelaide')]),
                html.Tr([html.Td(id='dist_darw'), html.Td(id='dist_ade')]),
            ]),
            html.Div([
                html.Table([
                    # Header with merged cells
                    html.Tr([html.Th('Next control stop', colSpan=2)]),
                    
                    # Remaining rows
                    html.Tr([html.Th(id='cs_name', colSpan=2)]),
                    html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                    html.Tr([html.Td(id='cs_lat'), html.Td(id='cs_lng')]),
                    html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
                    html.Tr([html.Td(id='cs_dist_dar'), html.Td(id='cs_dist_us')]),
                ])
            ])
        ])
    ], style={'display': 'flex', 'alignItems': 'center', 'padding-left': '20px', 'padding-right': '20px'}),


    # Container for the columns
    html.Div([
        # Map and static plots
        html.Div(children=[
            # Map
            dcc.Graph(id='map',
                config={'displayModeBar': 'hover'},
                figure={
                    'data': [{
                        'type': 'scattermapbox',
                        'lat': route_data['latitude'],
                        'lon': route_data['longitude'],
                        'mode': 'lines+markers',
                        'marker': {'size': 3},
                        'line': {'color': 'blue'},
                        'hoverinfo': 'none',
                        'name': 'Route'
                    },
                    {
                        'type': 'scattermapbox',
                        'lat': api_route_df['latitude'],
                        'lon': api_route_df['longitude'],
                        'mode': 'markers',
                        'marker': {'size': 3},
                        'line': {'color': 'white'},
                        'hoverinfo': 'none',
                        'name': 'Weather locations'
                    },
                    {
                        'type': 'scattermapbox',
                        'lat': control_stops_df['latitude'],
                        'lon': control_stops_df['longitude'],
                        'mode': 'markers',
                        'marker': {'size': 5},
                        'line': {'color': 'orange'},
                        'hovertext': (control_stops_df['town'] + ": " + control_stops_df['location']).to_list(),
                        'hoverinfo': 'text',
                        'name': 'Control Stops'
                    },
                    {
                        'type': 'scattermapbox',
                        'lat': [current_position['latitude']],
                        'lon': [current_position['longitude']],
                        'mode': 'markers',
                        'marker': {'size': 10, 'color': 'red'},
                        'name': 'Aletch',
                        'hovertext': 'Current position',
                        'hoverinfo': 'text'
                    }],
                    'layout': {
                        'mapbox': {
                            'style': "mapbox://styles/mapbox/satellite-streets-v11",
                            'accesstoken': constants.TOKEN_MAPBOX,
                            'center': {'lat': middle_lat, 'lon': middle_lng},
                            'zoom': 3.5,
                            'minzoom': 3.5
                        },
                        'margin': {'l': 0, 'r': 0, 'b': 0, 't': 0},
                        'legend': {
                            'x': 0,
                            'y': 1,
                            'xanchor': 'left',
                            'yanchor': 'top',
                            'bgcolor': 'transparent'
                        }
                    }
                }
            ),

            # Velocity
            html.Br(),
            dcc.Graph(id='plot_velocity',
                figure={
                    'data': [
                        {
                            'x': x_cumDistance_km,
                            'y': route_data['maxSpeed'],
                            'type': 'scatter',
                            'mode': 'lines',
                            'name': 'Max speed',
                            'hoverinfo': 'none',
                        },
                        {
                            'x': DP_data['cumDistance'] / 1000,
                            'y': DP_data['velocity'],
                            'type': 'scatter',
                            'mode': 'lines',
                            'name': 'DP long-term',
                            'hoverinfo': 'none',
                        }
                    ],
                    'layout': {
                        'margin': {
                            "r": 0,
                            "t": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'Velocity (km/h)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'legend': {
                        'x': 1,
                        'y': 1,
                        'xanchor': 'right',
                        'yanchor': 'top'},
                        'shapes': vertical_lines,
                        'height': height_plot
                    }
                }
            ),

            # SoC
            html.Br(),
            dcc.Graph(id='plot_soc',
                figure={
                    'data': [
                        {
                            'x': DP_data['cumDistance'] / 1000,
                            'y': DP_data['soc'],
                            'type': 'scatter',
                            'mode': 'lines',
                            'name': 'DP long-term',
                            'hoverinfo': 'none',
                        }
                    ],
                    'layout': {
                        'margin': {
                            "r": 0,
                            "t": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'SoC (%)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        # 'legend': {
                        # 'x': 1,
                        # 'y': 1,
                        # 'xanchor': 'right',
                        # 'yanchor': 'top'},
                        'shapes': vertical_lines,
                        'height': height_plot
                    }
                }
            ),

            # Altitude
            html.Br(),
            dcc.Graph(id='plot_altitude',
                figure={
                    'data': [
                        {
                            'x': x_cumDistance_km,
                            'y': route_data['altitudeSmoothed'],
                            'type': 'scatter',
                            'mode': 'lines',
                            'hoverinfo': 'none',
                        },
                    ],
                    'layout': {
                        'margin': {
                            "r": 0,
                            "t": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'Altitude (m)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'shapes': vertical_lines,
                        'height': height_plot
                    }
                }
            ),

            # Inclination
            html.Br(),
            dcc.Graph(id='plot_inclination',
                figure={
                    'data': [
                        {
                            'x': x_cumDistance_km,
                            'y': route_data['inclinationSmoothed'].apply(rad2deg),
                            'type': 'scatter',
                            'mode': 'lines',
                            'hoverinfo': 'none',
                        },
                    ],
                    'layout': {
                        'margin': {
                            "r": 0,
                            "t": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'Inclination (°)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'shapes': vertical_lines,
                        'height': height_plot
                    }
                }
            )
        ], style={'padding': 10, 'flex': 1}),

        # Time slider
        html.Div(
            dcc.RangeSlider(
                id='rangeSlider_hour_minute',
                min=0,
                max=len(unique_days) - 1,
                value=[0, len(unique_days) - 1],
                marks=slider_marks,
                vertical=True,
                step=1,
                verticalHeight=height_slider
            ),
            style={'width': '70px', 'height':height_slider},
            id='rangeSlider_hour_minute-container'
        ),

        # Weather plots
        html.Div(children=[
            dcc.Graph(id='plot_globalIrradiance',
                figure=initialize_plot('Global irradiance (W/m²)')),
            dcc.Graph(id='plot_temperature',
                figure=initialize_plot('Temperature (°)')),
            dcc.Graph(id='plot_wind',
                figure=initialize_plot('Wind (km/h)')),
            dcc.Graph(id='plot_precipitation',
                figure=initialize_plot('Precipitation (mm)')),

        ], style={'padding': 10, 'flex': 1})
    ], style={'display': 'flex', 'flex-direction': 'row'})
    
], style={'display': 'flex', 'flex-direction': 'column'})

### Callbacks
# Last update forecasts
@app.callback(
    Output('last_update_sf', 'children'),
    Output('last-update-cm', 'children'),
    Output('last_update_sc', 'children'),
    Input('interval_update', 'n_intervals')
)
def update_output_div(n):
    time_SF_str = api_requester.get_solar_forecast_last_time
    if time_SF_str is pd.NaT:
        print_SF = "Last update SF: No data available"
    else:
        print_SF = f"Last update SF: {time_SF_str.strftime('%d %B %Y at %H:%M')}"

    time_CM_str = api_requester.get_solar_forecast_cloudmove_last_time
    if time_CM_str is pd.NaT:
        print_CM = "Last update CM: No data available"
    else:
        print_CM = f"Last update CM: {time_CM_str.strftime('%d %B %Y at %H:%M')}"

    time_SC_str = solcast_executer.get_solcast_last_time
    if time_SC_str is pd.NaT:
        print_SC = "Last update SC: No data available"
    else:
        print_SC = f"Last update SC: {time_SC_str.strftime('%d %B %Y at %H:%M')}"

    return print_SF, print_CM, print_SC

# Time slider callback
@app.callback(
    Output('rangeSlider_hour_minute', 'max'),
    Output('rangeSlider_hour_minute', 'marks'),
    Output('rangeSlider_hour_minute', 'value'),
    Input('dropdown_day_month_year', 'value'),
    Input('interval_update', 'n_intervals'))
def update_slider(day_month_year, n):
    min_value = 0
    now = pd.Timestamp.now(tz=constants.TIMEZONE)
    
    unique_times_days = unique_times[unique_times.map(lambda x: x.strftime('%d %B %Y')) == day_month_year]
    slider_marks = {i: {'label': time.strftime('%H:%M')} for i, time in enumerate(unique_times_days)}

    next_time_index = unique_times_days.searchsorted(now)
    if next_time_index < len(unique_times_days) and now.day == pd.to_datetime(day_month_year).day:
        slider_marks[next_time_index]['style'] = {'color': 'red'}
        min_value = next_time_index

    #TODO If 17 is present, update max_value to that value
    # Otherwise
    max_value = len(unique_times_days) - 1
    value = [min_value, max_value]

    return max_value, slider_marks, value

# Axis callback
@app.callback(
    [Output('plot_velocity', 'figure', allow_duplicate=True),
     Output('plot_soc', 'figure', allow_duplicate=True),
     Output('plot_altitude', 'figure', allow_duplicate=True),
     Output('plot_inclination', 'figure', allow_duplicate=True),
     Output('plot_globalIrradiance', 'figure', allow_duplicate=True),
     Output('plot_temperature', 'figure', allow_duplicate=True),
     Output('plot_wind', 'figure', allow_duplicate=True),
     Output('plot_precipitation', 'figure', allow_duplicate=True)],
    [Input('plot_velocity', 'relayoutData'),
     Input('plot_soc', 'relayoutData'),
     Input('plot_altitude', 'relayoutData'),
     Input('plot_inclination', 'relayoutData'),
     Input('plot_globalIrradiance', 'relayoutData'),
     Input('plot_temperature', 'relayoutData'),
     Input('plot_wind', 'relayoutData'),
     Input('plot_precipitation', 'relayoutData')],
    [State('plot_velocity', 'figure'),
     State('plot_soc', 'figure'),
     State('plot_altitude', 'figure'),
     State('plot_inclination', 'figure'),
     State('plot_globalIrradiance', 'figure'),
     State('plot_temperature', 'figure'),
     State('plot_wind', 'figure'),
     State('plot_precipitation', 'figure')],
    prevent_initial_call=True)
def update_xaxis(v, soc, h, dh, gh, T, w, rr, fig_v, fig_soc, fig_h, fig_dh, fig_gh, fig_T, fig_w, fig_rr):
    # Get the callback context to determine which input triggered the callback
    ctx = dash.callback_context
    if not ctx.triggered:
        raise dash.exceptions.PreventUpdate
    input_id = ctx.triggered[0]['prop_id'].split('.')[0]

    trigger_value = ctx.triggered[0]['value']

    # PreventUpdate when autosize
    if 'autosize' in trigger_value and trigger_value['autosize']:
        if trigger_value['autosize']:
            raise dash.exceptions.PreventUpdate
    
    # PreventUpdate while dragging
    elif 'dragmode' in trigger_value and trigger_value['dragmode'] == 'pan':
            raise dash.exceptions.PreventUpdate
    
    # PreventUpdate when changing yxis only
    elif ('yaxis.range[0]' in trigger_value or 'yaxis.range[1]' in trigger_value) and not ('xaxis.range[0]' in trigger_value or 'xaxis.range[1]' in trigger_value):
        raise dash.exceptions.PreventUpdate
    
    # If autorange is set for any graph, let all graphs determine their x-axis range automatically
    elif 'xaxis.autorange' in trigger_value and trigger_value['xaxis.autorange']:
        fig_v['layout']['xaxis']['autorange'] = True
        fig_soc['layout']['xaxis']['autorange'] = True
        fig_h['layout']['xaxis']['autorange'] = True
        fig_dh['layout']['xaxis']['autorange'] = True
        fig_gh['layout']['xaxis']['autorange'] = True
        fig_T['layout']['xaxis']['autorange'] = True
        fig_w['layout']['xaxis']['autorange'] = True
        fig_rr['layout']['xaxis']['autorange'] = True

    else:
        # Determine the new x-axis range
        if input_id == 'plot_velocity':
            xaxis_range = [v['xaxis.range[0]'], v['xaxis.range[1]']]
        elif input_id == 'plot_soc':
            xaxis_range = [soc['xaxis.range[0]'], soc['xaxis.range[1]']]
        elif input_id == 'plot_altitude':
            xaxis_range = [h['xaxis.range[0]'], h['xaxis.range[1]']]
        elif input_id == 'plot_inclination':
            xaxis_range = [dh['xaxis.range[0]'], dh['xaxis.range[1]']]
        elif input_id == 'plot_globalIrradiance':
            xaxis_range = [gh['xaxis.range[0]'], gh['xaxis.range[1]']]
        elif input_id == 'plot_temperature':
            xaxis_range = [T['xaxis.range[0]'], T['xaxis.range[1]']]
        elif input_id == 'plot_wind':
            xaxis_range = [w['xaxis.range[0]'], w['xaxis.range[1]']]
        elif input_id == 'plot_precipitation':
            xaxis_range = [rr['xaxis.range[0]'], rr['xaxis.range[1]']]
        else:
            raise dash.exceptions.PreventUpdate

        # Update the x-axis range for all figures and set autorange to False
        fig_v['layout']['xaxis']['range'] = xaxis_range
        fig_v['layout']['xaxis']['autorange'] = False

        fig_soc['layout']['xaxis']['range'] = xaxis_range
        fig_soc['layout']['xaxis']['autorange'] = False

        fig_h['layout']['xaxis']['range'] = xaxis_range
        fig_h['layout']['xaxis']['autorange'] = False

        fig_dh['layout']['xaxis']['range'] = xaxis_range
        fig_dh['layout']['xaxis']['autorange'] = False

        fig_gh['layout']['xaxis']['range'] = xaxis_range
        fig_gh['layout']['xaxis']['autorange'] = False

        fig_T['layout']['xaxis']['range'] = xaxis_range
        fig_T['layout']['xaxis']['autorange'] = False

        fig_w['layout']['xaxis']['range'] = xaxis_range
        fig_w['layout']['xaxis']['autorange'] = False

        fig_rr['layout']['xaxis']['range'] = xaxis_range
        fig_rr['layout']['xaxis']['autorange'] = False

    return fig_v, fig_soc, fig_h, fig_dh, fig_gh, fig_T, fig_w, fig_rr

#BUG SOMETIMES THE RED LINES DOES NOT UPDATE
# Weather plots
@app.callback(
    Output('plot_globalIrradiance', 'figure'),
    Output('plot_temperature', 'figure'),
    Output('plot_wind', 'figure'),
    Output('plot_precipitation', 'figure'),
    Input('dropdown_day_month_year', 'value'),
    Input('rangeSlider_hour_minute', 'value'),
    Input('rangeSlider_hour_minute', 'marks'),
    State('plot_globalIrradiance', 'figure'),
    State('plot_temperature', 'figure'),
    State('plot_wind', 'figure'),
    State('plot_precipitation', 'figure'),
    State('store_current_position', 'data'))
def update_graphs(day_month_year, hour_minute, slider_marks, fig_gh, fig_tt, fig_ww, fig_rr, current_position):
    # Extract timezone from the DataFrame's index (should be constants.TIMEZONE)
    timezone = preprocessed_df.index.get_level_values('time').tz 

    # Create the start and end timestamps for the day
    start_time = pd.Timestamp(day_month_year).tz_localize(timezone)
    end_time = start_time + pd.Timedelta(days=1)

    # Filter the DataFrame for the specific day
    data_for_selected_day = preprocessed_df[(preprocessed_df.index.get_level_values('time') >= start_time) & 
                            (preprocessed_df.index.get_level_values('time') < end_time)]
    
    # Create the start and end timestamps for the range
    first_datetime_str = day_month_year + " " + slider_marks[f'{hour_minute[0]}']['label'] # Slider marks are str and hour_minute int
    first_datetime_obj = pd.Timestamp(first_datetime_str).tz_localize(timezone)
    last_datetime_str = day_month_year + " " + slider_marks[f'{hour_minute[1]}']['label'] # Slider marks are str and hour_minute int
    last_datetime_obj = pd.Timestamp(last_datetime_str).tz_localize(timezone)

    # Create a mask to select the data in the range
    mask = (data_for_selected_day.index.get_level_values('time') >= first_datetime_obj) & (data_for_selected_day.index.get_level_values('time') <= last_datetime_obj)
    data_in_range_df = data_for_selected_day[mask]

    x_cumDistance_range_km = data_in_range_df.index.get_level_values('cumDistance').unique() / 1000

    fig_gh['layout']['shapes'][11]['x0'] = current_position['cumDistance_km']
    fig_gh['layout']['shapes'][11]['x1'] = current_position['cumDistance_km']
    fig_gh['data'] = [
        go.Scatter(
            x=x_cumDistance_range_km,
            y=data_in_range_df['globalIrradiance'].groupby(level='cumDistance').mean(),
            mode='lines',
            hoverinfo='none'
        )
    ]
    
    fig_tt['layout']['shapes'][11]['x0'] = current_position['cumDistance_km']
    fig_tt['layout']['shapes'][11]['x1'] = current_position['cumDistance_km']
    fig_tt['data'] = [
        go.Scatter(
            x=x_cumDistance_range_km,
            y=data_in_range_df['temperature'].groupby(level='cumDistance').mean(),
            mode='lines',
            hoverinfo='none'
        )
    ]

    fig_ww['layout']['shapes'][11]['x0'] = current_position['cumDistance_km']
    fig_ww['layout']['shapes'][11]['x1'] = current_position['cumDistance_km']
    # Compute series for each trace
    wind_speed_series = data_in_range_df['windSpeed'].groupby(level='cumDistance').mean()
    side_wind_series = abs(data_in_range_df['sideWind'].groupby(level='cumDistance').mean())
    front_wind_series = data_in_range_df['frontWind'].groupby(level='cumDistance').mean()

    # Combine series into a single dataframe
    combined_df = pd.concat({
        'Wind speed': wind_speed_series,
        'Side wind': side_wind_series,
        'Front wind': front_wind_series
    }, axis=1)

    # Create traces for wind data
    wind_traces = []

    for column in combined_df.columns:
        trace = go.Scatter(
            x=x_cumDistance_range_km,
            y=combined_df[column],
            mode='lines',
            name=column,
            hoverinfo='none'
        )
        wind_traces.append(trace)

    fig_ww['data'] = wind_traces

    fig_rr['layout']['shapes'][11]['x0'] = current_position['cumDistance_km']
    fig_rr['layout']['shapes'][11]['x1'] = current_position['cumDistance_km']
    fig_rr['data'] = [
        go.Scatter(
            x=x_cumDistance_range_km,
            y=data_in_range_df['precipitation'].groupby(level='cumDistance').mean(),
            mode='lines',
            hoverinfo='none'
        )
    ]

    return fig_gh, fig_tt, fig_ww, fig_rr

## CURRENT POSITION
# Store GPS data
@app.callback(
    Output('store_current_position', 'data'),
    Input('interval_update', 'n_intervals'))
def update_current_position(n):
    
    random_number = random.randint(300, 10000)
    current_position = {
        'latitude': route_data['latitude'][random_number],
        'longitude': route_data['longitude'][random_number]
    }
    # current_position = gps.get_current_location()
    current_cumDistance = route.find_closest_row(current_position)[0]['cumDistance']
    current_position['cumDistance'] = current_cumDistance
    current_position['cumDistance_km'] = current_cumDistance / 1000

    if current_position is None:
        raise dash.exceptions.PreventUpdate
    
    return current_position

# Show data in box and lines in map and weather plots
@app.callback(
    Output('current_lat', 'children'),
    Output('current_lng', 'children'),
    Output('dist_darw', 'children'),
    Output('dist_ade', 'children'),
    Output('map', 'figure'),
    Output('plot_velocity', 'figure'),
    Output('plot_soc', 'figure'),
    Output('plot_altitude', 'figure'),
    Output('plot_inclination', 'figure'),
    Input('store_current_position', 'data'),
    State('map', 'figure'),
    State('plot_velocity', 'figure'),
    State('plot_soc', 'figure'),
    State('plot_altitude', 'figure'),
    State('plot_inclination', 'figure'))
def plots_current_position(current_position, fig_map, fig_v, fig_soc, fig_h, fig_dh):
    fig_map['data'][3]['lat'] = [current_position['latitude']]
    fig_map['data'][3]['lon'] = [current_position['longitude']]

    fig_v['layout']['shapes'][11]['x0'] = current_position['cumDistance_km']
    fig_v['layout']['shapes'][11]['x1'] = current_position['cumDistance_km']

    fig_soc['layout']['shapes'][11]['x0'] = current_position['cumDistance_km']
    fig_soc['layout']['shapes'][11]['x1'] = current_position['cumDistance_km']

    fig_h['layout']['shapes'][11]['x0'] = current_position['cumDistance_km']
    fig_h['layout']['shapes'][11]['x1'] = current_position['cumDistance_km']

    fig_dh['layout']['shapes'][11]['x0'] = current_position['cumDistance_km']
    fig_dh['layout']['shapes'][11]['x1'] = current_position['cumDistance_km']

    return current_position['latitude'], current_position['longitude'], round(current_position['cumDistance_km'], 1), round(max_cumDistance_km - current_position['cumDistance_km'], 1), fig_map, fig_v, fig_soc, fig_h, fig_dh

# Control stops
@app.callback(
    [Output('cs_lat', 'children'),
     Output('cs_lng', 'children'),
     Output('cs_dist_dar', 'children'),
     Output('cs_dist_us', 'children'),
     Output('cs_name', 'children')],
    Input('store_current_position', 'data'))
def next_control_stop(current_position):
    row_route, _ = route.find_closest_row(current_position)
    current_cum_distance_km = row_route['cumDistance'] / 1000

    row, _ = route.find_next_cs(current_position, print_is_requested=False)
    cs_cum_distance_km = row['cumDistance'] / 1000
    name = row['town'] + ': ' + row['location']

    delta_km = cs_cum_distance_km - current_cum_distance_km

    return row['latitude'], row['longitude'], round(cs_cum_distance_km, 1), round(delta_km, 1), name

## OVERNIGHT STOP
# Overnight stops
@app.callback(
    Output('os_max_v_lat', 'children'),
    Output('os_max_v_lng', 'children'),
    Output('os_max_v_dist_darw', 'children'),
    Output('os_max_v_dist_us', 'children'),
    Input('store_current_position', 'data'))
def next_overnight_stop(current_position):
    os_cumDistance = update_max_speed_distance(current_position)

    # PreventUpdate if we are already in the overnight stop
    if os_cumDistance is None:
        raise dash.exceptions.PreventUpdate
    
    row, _ = route.find_closest_row_cumDistance(os_cumDistance)
    os_cum_distance_km = os_cumDistance / 1000
    # os_cum_dist_km2 = row['cumDistance'] / 1000
    # print(os_cum_dist_km, os_cum_dist_km2)

    delta_km = os_cum_distance_km - current_position['cumDistance_km']

    return row['latitude'], row['longitude'], round(os_cum_distance_km, 1), round(delta_km, 1)

# Submit button cumDistance
@app.callback(
    Output('submit_cum_dist', 'disabled'),
    Output('input_cum_dist', 'min'),
    Input('input_cum_dist', 'value'),
    State('store_current_position', 'data'))
def update_button_cumDistance(input_cumDistance_km, current_position):
    if input_cumDistance_km is not None and current_position['cumDistance_km'] <= input_cumDistance_km <= max_cumDistance_km:
        return False, current_position['cumDistance_km']
    return True, current_position['cumDistance_km']

# Output given a cumDistance
@app.callback(
    Output('output_cumDistance', 'children'),
    Input('submit_cum_dist', 'n_clicks'),
    State('input_cum_dist', 'value'),
    State('store_current_position', 'data'))
def update_output_cumDistance(n_clicks, input_cumDistance_km, current_position):

    if n_clicks:
        # If smaller than current cumDistance
        if input_cumDistance_km < current_position['cumDistance_km']:
            return 'Error: position before current position'
        
        row, _ = route.find_closest_row_cumDistance(input_cumDistance_km * 1000)
        lat = row['latitude']
        lng = row['longitude']
        
        # Calculate the time difference between now and 17:00 of the current day
        now = pd.Timestamp.now()
        target_time = now.replace(hour=17, minute=0, second=0, microsecond=0)
        delta_hour = (target_time - now).seconds / 3600

        # Check if the current time is already past 17:00
        if delta_hour < 0:
            return 'Error: The current time is past 17:00'
        
        mean_v = (input_cumDistance_km - current_position['cumDistance_km']) / delta_hour
        return f'{lat}, {lng} Mean velocity: {round(mean_v, 1)} km/h'

# Submit button latitude and longitude
@app.callback(
    Output('submit_lat_lng', 'disabled'),
    Input('input_lat', 'value'),
    Input('input_lng', 'value'))
def update_button_latlon(input_lat, input_lng):
    if input_lat is not None and constants.GEO['latitude']['min'] <= input_lat <= constants.GEO['latitude']['max'] and input_lng is not None and constants.GEO['longitude']['min'] <= input_lng <= constants.GEO['longitude']['max']:
        return False
    return True

# Output given latitude and longitude
@app.callback(
    Output('output_lat_lng', 'children'),
    Input('submit_lat_lng', 'n_clicks'),
    State('input_lat', 'value'),
    State('input_lng', 'value'),
    State('store_current_position', 'data'))
def update_output_latlon(n_clicks, input_lat, input_lng, current_position):
    if n_clicks:
        if input_lat is None or not (constants.GEO['latitude']['min'] <= input_lat <= constants.GEO['latitude']['max']) or input_lng is None or not (constants.GEO['longitude']['min'] <= input_lng <= constants.GEO['longitude']['max']):
            return 'Error: One or both values are out of range!'
        
        position = {'latitude': input_lat, 'longitude': input_lng}
        row, _ = route.find_closest_row(position)
        position_cumDistance_km = row['cumDistance'] / 1000

        # If smaller than current cumDistance
        if position_cumDistance_km <  current_position['cumDistance_km']:
            return 'Error: Input position before current position'
        
        # Calculate the time difference between now and 17:00 of the current day
        now = pd.Timestamp.now()
        target_time = now.replace(hour=17, minute=0, second=0, microsecond=0)
        delta_hour = (target_time - now).seconds / 3600

        # Check if the current time is already past 17:00
        if delta_hour < 0:
            return 'Error: The current time is past 17:00'
        
        mean_v = (position_cumDistance_km -  current_position['cumDistance_km']) / delta_hour
        return f'{round(position_cumDistance_km, 2)} Mean velocity: {round(mean_v, 1)} km/h'

## DRIVER STOP
# Dropdown time driver stop
@app.callback(
    Output('dropdown_ds_minute', 'options'),
    Input('dropdown_ds_hour', 'value')
)
def update_minutes(selected_hour):
    if selected_hour == current_hour:
        return [{'label': str(i), 'value': i} for i in range(current_minute, 60)]
    else:
        return [{'label': str(i), 'value': i} for i in range(60)]

# Submit button driver stop time
@app.callback(
    Output('submit_time_ds', 'disabled'),
    Input('dropdown_ds_hour', 'value')
)
def enable_button(hour):
    return hour >= 17  # If time is 17:00 or later, button is disabled

# Output time driver stop
@app.callback(
    Output('output_time', 'children'),
    Output('store_driver_stop', 'data'),
    [Input('submit_time_ds', 'n_clicks')],
    [State('dropdown_ds_hour', 'value'),
     State('dropdown_ds_minute', 'value')]
)
def display_time(n_clicks, hour, minute):
    if n_clicks is None:
        return dash.no_update, dash.no_update
    return f"You selected {hour:02}:{minute:02}", {'hour': hour, 'minute': minute}

# Driver stop
@app.callback(
    Output('ds_max_v_lat', 'children'),
    Output('ds_max_v_lng', 'children'),
    Output('ds_max_v_dist_darw', 'children'),
    Output('ds_max_v_dist_us', 'children'),
    Input('store_current_position', 'data'),
    Input('store_driver_stop', 'data'))
def next_overnight_stop(current_position, time):
    # PreventUpdate if time is an empty dict
    if not time:
        raise dash.exceptions.PreventUpdate
    
    os_cumDistance = update_max_speed_distance(current_position, time['hour'], time['minute'])

    # PreventUpdate if we are already in the overnight stop
    if os_cumDistance is None:
        raise dash.exceptions.PreventUpdate
    
    row, _ = route.find_closest_row_cumDistance(os_cumDistance)
    os_cum_distance_km = os_cumDistance / 1000
    # os_cum_dist_km2 = row['cumDistance'] / 1000
    # print(os_cum_dist_km, os_cum_dist_km2)

    delta_km = os_cum_distance_km - current_position['cumDistance_km']

    return row['latitude'], row['longitude'], round(os_cum_distance_km, 1), round(delta_km, 1)


if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="tab")


Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

--- Recursive call ---
--- Recursive call ---
All control stops considered
--- Recursive call ---
All control stops considered
--- Recursive call ---
All control stops considered
--- Recursive call ---
--- Recursive call ---
All control stops considered
--- Recursive call ---
All control stops considered
--- Recursive call ---
--- Recursive call ---
--- Recursive call ---
All control stops considered
All control stops considered
--- Recursive call ---
--- Recursive call ---
All control stops considered
--- Recursive call ---
All control stops considered
--- Recursive call ---
--- Recursive call ---
--- Recursive call ---
All control stops considered
All control stops considered
All control stops considered
--- Recursive call ---
All control stops considered
--- Recursive call ---
--- Recursive call ---
--- Recursive call ---
All control stops considered
All control stops considered
--- Recursive call ---
--- Recursive call ---
--- Recursive call ---
All control stops considered
All con

In [14]:
def _recursive_position_finder(current_cumDistance:float, driving_time:float, cs_to_skip:int, type:str) -> pd.Series:
    """ """
    type_dict = {
        'max_speed': {'var': 'cumTimeAtMaxSpeedLim', 'data': route_data},
        # 'opt_speed': {'var': 'velocity', 'data': DP_data},
        'avg_speed': {'var': 'velocity', 'data': cumTimeAtMeanVelocity(50)},
    }

    # Cut data at current position (lower cut)
    cut_data = type_dict[type]['data'].copy()

    cut_data = cut_data[cut_data['cumDistance'] >= current_cumDistance]
    cut_data = cut_data.reset_index(drop=True)

    current_time = cut_data[type_dict[type]['var']][0]

    # Cut data at driving time (upper cut)
    cut_data = cut_data[cut_data[type_dict[type]['var']] <= current_time + driving_time]
    max_cumDistance = cut_data['cumDistance'].max()

    # Check if the control stop dataframe is not empty
    if not control_stops.empty:
        cs_in_range_mask = (control_stops['cumDistance'] >= current_cumDistance) & (control_stops['cumDistance'] <= max_cumDistance)
        cs_in_range = cs_in_range_mask.sum()
        print(f'cs found ahead: {cs_in_range}')
        print(f'cs to skip: {cs_to_skip}')
    else:
        print("No control stop dataframe given")

    # Stop cases
    # Reach end of route, return last point
    if current_cumDistance >= route_data.iloc[-1]['cumDistance']:
        return route_data.iloc[-1] # return self.end_position
    
    # All control stops considered
    if cs_to_skip == cs_in_range:
        print("All control stops considered")
        return route_data.loc[route_data['cumDistance'] == max_cumDistance].iloc[0]
    
    # Stop at control stop for the night, meaning we arrive at cs between 16:30 and 17:00
    if cs_to_skip > cs_in_range:
        print("Stop at control stop for the night")
        return control_stops.loc[control_stops['cumDistance'] > current_cumDistance].iloc[cs_to_skip - 1]
    

    # Recursive call to skip control stop and reduce driving time by 30 minutes
    if cs_to_skip < cs_in_range: # Case of 0 cs in range considered
        print("--- Recursive call ---")
        return _recursive_position_finder(current_cumDistance, driving_time - 30.0*60.0, cs_to_skip + 1, type)
    
def update_max_speed_distance(current_position:dict, time:dict, type:str) -> float:
    """ """
    # Subtract overnight stop start time to now
    now = pd.Timestamp.now()
    driving_time = pd.Timedelta(hours=time['hours'], minutes=time['minutes']) - pd.Timedelta(hours=now.hour, minutes=now.minute)
    current_cumDistance = route.find_closest_row(current_position, print_is_requested=False)[0]['cumDistance']

    position_series = _recursive_position_finder(current_cumDistance, driving_time.total_seconds(), cs_to_skip=0, type=type, i=0)

    return position_series['cumDistance']

app = Dash(__name__)

app.layout = html.Div([
    dcc.Interval(id='interval_update', interval=update_sec*1000, n_intervals=0),
    html.Table([
        # Header with merged cells
        html.Tr([html.Th('Next overnight stop', colSpan=2)]),
        
        # Remaining rows
        html.Tr([html.Th('At max speed', colSpan=2)]),
        html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
        html.Tr([html.Td(id='os_max_v_lat'), html.Td(id='os_max_v_lng')]),
        html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
        html.Tr([html.Td(id='os_max_v_dist_darw'), html.Td(id='os_max_v_dist_us')]),
    ])
])

@app.callback(
    [Output('os_max_v_lat', 'children'),
     Output('os_max_v_lng', 'children'),
     Output('os_max_v_dist_darw', 'children'),
     Output('os_max_v_dist_us', 'children')],
    Input('interval_update', 'n_intervals')
)
def next_overnight_stop(n_intervals):
    # Get the current position
    # current_position = route.get_current_position() #TODO CHANGE WITH GPS

    row_route, _ = route.find_closest_row(current_position)
    current_cum_dist_km = row_route['cumDistance'] / 1000

    type = 'max_speed'

    os_cum_dist = update_max_speed_distance(current_position, time={'hours': 17, 'minutes': 0}, type=type)
    row, _ = route.find_closest_row_cumDistance(os_cum_dist)
    os_cum_dist_km = os_cum_dist / 1000
    os_cum_dist_km2 = row['cumDistance'] / 1000
    print(os_cum_dist_km, os_cum_dist_km2)

    delta_dis = os_cum_dist_km - current_cum_dist_km

    return row['latitude'], row['longitude'], round(os_cum_dist_km, 1), round(delta_dis, 1)

if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="tab")


Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

0
cs found ahead: 1
cs to skip: 0
--- Recursive call ---
1
cs found ahead: 1
cs to skip: 1
All control stops considered
459.357237472554 459.357237472554


In [15]:
app = dash.Dash(__name__)

# Get the current time
now = pd.Timestamp.now()
current_hour = now.hour
current_minute = now.minute

# Create hour and minute options based on constraints
hour_options = [{'label': str(i), 'value': i} for i in range(current_hour, 18)]
minute_options = [{'label': str(i), 'value': i} for i in range(current_minute, 60)] if current_hour == now.hour else [{'label': str(i), 'value': i} for i in range(60)]

app.layout = html.Div([
    dcc.Dropdown(
        id='dropdown_ds_hour',
        options=hour_options,
        value=current_hour,
        clearable=False
    ),
    dcc.Dropdown(
        id='dropdown_ds_minute',
        options=minute_options,
        value=current_minute,
        clearable=False
    ),
    html.Button('Submit', id='submit_time_ds', disabled=True),
    html.Div(id='output_time')
])

@app.callback(
    Output('dropdown_ds_minute', 'options'),
    Input('dropdown_ds_hour', 'value')
)
def update_minutes(selected_hour):
    if selected_hour == current_hour:
        return [{'label': str(i), 'value': i} for i in range(current_minute, 60)]
    else:
        return [{'label': str(i), 'value': i} for i in range(60)]

@app.callback(
    Output('submit_time_ds', 'disabled'),
    Input('dropdown_ds_hour', 'value')
)
def enable_button(hour):
    return hour >= 17  # If time is 17:00 or later, button is disabled

@app.callback(
    Output('output_time', 'children'),
    [Input('submit_time_ds', 'n_clicks')],
    [State('dropdown_ds_hour', 'value'),
     State('dropdown_ds_minute', 'value')]
)
def display_time(n_clicks, hour, minute):
    if n_clicks is None:
        return dash.no_update  # No updates until the button is clicked
    return f"You selected {hour:02}:{minute:02}"

if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="tab")


Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

In [ ]:
    # figure_gh['data'][0]['x'] = x_cumDistance_range_km
    # figure_gh['data'][0]['y'] = data_in_range_df['globalIrradiance'].groupby(level='cumDistance').mean()
    # line_shape = [{
    #             'type': 'line',
    #             'x0': x_cumDistance_range_km.min(),
    #             'x1': x_cumDistance_range_km.max(),
    #             'y0': data_for_selected_day['globalIrradiance'].max(),
    #             'y1': data_for_selected_day['globalIrradiance'].max(),
    #             'line': {
    #                 'color': 'red',
    #                 'width': 1}
    #             },
    #         ]
    # figure_gh['layout']['shapes'] = vertical_lines + line_shape

    # figure_gh={
    #     'data': [{
    #         'x': x_cumDistance_range_km,
    #         'y': data_in_range_df['globalIrradiance'].groupby(level='cumDistance').mean(),
    #         'type': 'scatter',
    #         'mode': 'lines',
    #         'hoverinfo': 'none'}],
    #     'layout': {
    #         'margin': {"r": 0,
    #                    "t": 0},
    #         'xaxis': {
    #             'title': x_cumDistance_str,
    #             'showgrid': True,
    #             'gridcolor': grid_color,
    #             'showline': True,
    #             'linecolor': box_color},
    #         'yaxis': {
    #             'title': 'Global irradiance (W/m²)',
    #             'showgrid': True,
    #             'gridcolor': grid_color,
    #             'showline': True,
    #             'linecolor': box_color},
    #         'shapes': [{
    #             'type': 'line',
    #             'x0': x_cumDistance_range_km.min(),
    #             'x1': x_cumDistance_range_km.max(),
    #             'y0': data_for_selected_day['globalIrradiance'].max(),
    #             'y1': data_for_selected_day['globalIrradiance'].max(),
    #             'line': {
    #                 'color': 'red',
    #                 'width': 1}
    #             },
    #         ] + vertical_lines,
    #         'height': height_plot
    #     }
    # }

    # figure_tt={
    #     'data': [{
    #         'x': x_cumDistance_range_km,
    #         'y': data_in_range_df['temperature'].groupby(level='cumDistance').mean(),
    #         'type': 'scatter',
    #         'mode': 'lines',
    #         'hoverinfo': 'none'}],
    #     'layout': {
    #         'margin': {"r": 0,
    #                    "t": 0},
    #         'xaxis': {
    #             'title': x_cumDistance_str,
    #             'showgrid': True,
    #             'gridcolor': grid_color,
    #             'showline': True,
    #             'linecolor': box_color},
    #         'yaxis': {
    #             'title': 'Temperature (°C)',
    #             'showgrid': True,
    #             'gridcolor': grid_color,
    #             'showline': True,
    #             'linecolor': box_color},
    #         'shapes': [{
    #             'type': 'line',
    #             'x0': x_cumDistance_range_km.min(),
    #             'x1': x_cumDistance_range_km.max(),
    #             'y0': data_for_selected_day['temperature'].max(),
    #             'y1': data_for_selected_day['temperature'].max(),
    #             'line': {
    #                 'color': 'red',
    #                 'width': 1}
    #             },
    #             {
    #             'type': 'line',
    #             'x0': x_cumDistance_range_km.min(),
    #             'x1': x_cumDistance_range_km.max(),
    #             'y0': data_for_selected_day['temperature'].min(),
    #             'y1': data_for_selected_day['temperature'].min(),
    #             'line': {
    #                 'color': 'blue',
    #                 'width': 1}
    #             }
    #         ] + vertical_lines,
    #         'height': height_plot
    #     }
    # }

    # figure_ww={
    #     'data': [{
    #         # 'x': x_cumDistance_range_km,
    #         # 'y': data_in_range_df['windGust'].groupby(level='cumDistance').mean(),
    #         # 'type': 'scatter',
    #         # 'mode': 'lines',
    #         # 'name': 'Wind gust'},
    #         # {
    #         'x': x_cumDistance_range_km,
    #         'y': data_in_range_df['windSpeed'].groupby(level='cumDistance').mean(),
    #         'type': 'scatter',
    #         'mode': 'lines',
    #         'name': 'Wind speed',
    #         'hoverinfo': 'none'},
    #         {
    #         'x': x_cumDistance_range_km,
    #         'y': abs(data_in_range_df['sideWind'].groupby(level='cumDistance').mean()),
    #         'type': 'scatter',
    #         'mode': 'lines',
    #         'name': 'Side wind',
    #         'hoverinfo': 'none'},
    #         {
    #         'x': x_cumDistance_range_km,
    #         'y': data_in_range_df['frontWind'].groupby(level='cumDistance').mean(),
    #         'type': 'scatter',
    #         'mode': 'lines',
    #         'name': 'Front wind',
    #         'hoverinfo': 'none',},
    #         ],
    #     'layout': {
    #         'margin': {"r": 0,
    #                    "t": 0},
    #         'xaxis': {
    #             'title': x_cumDistance_str,
    #             'showgrid': True,
    #             'gridcolor': grid_color,
    #             'showline': True,
    #             'linecolor': box_color},
    #         'yaxis': {
    #             'title': 'Wind (km/h)',
    #             'showgrid': True,
    #             'gridcolor': grid_color,
    #             'showline': True,
    #             'linecolor': box_color},
    #         'height': height_plot,
    #         'legend': {
    #             'x': 1,
    #             'y': 1,
    #             'xanchor': 'right',
    #             'yanchor': 'top'},
    #         'shapes': vertical_lines
    #     }
    # }

    # figure_rr={
    #     'data': [{
    #         'x': x_cumDistance_range_km,
    #         'y': round(data_in_range_df['precipitation'].groupby(level='cumDistance').mean(), 2),
    #         'type': 'scatter',
    #         'mode': 'lines',
    #         'hoverinfo': 'none',}],
    #     'layout': {
    #         'margin': {"r": 0,
    #                    "t": 0},
    #         'xaxis': {
    #             'title': x_cumDistance_str,
    #             'showgrid': True,
    #             'gridcolor': grid_color,
    #             'showline': True,
    #             'linecolor': box_color},
    #         'yaxis': {
    #             'title': 'Precipitation (mm)',
    #             'showgrid': True,
    #             'gridcolor': grid_color,
    #             'showline': True,
    #             'linecolor': box_color},
    #         'shapes': vertical_lines,
    #         'height': height_plot,
    #         'legend': {
    #             'x': 0.2,
    #             'y': 1,
    #             'xanchor': 'left',
    #             'yanchor': 'top'
    #         }
    #     }
    # }